In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [2]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    elif t=='each':
        x=x.dropna().values
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [3]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [4]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [5]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [6]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [59]:
holdings='AMZN,MELI,AAPL,FSLY,PTON,FUTU,BLCN,ZM,DOCU,NOMD,GDXJ,BGS,AWK,SHW,EAT,CF,BATT,CAKE,UAA,TTEK,NWL,FAST,FXE,DVA,URA,ITB,GOLD,SPY,DOCU,ZM,PTON,MSFT,NVDA,EA,CRM,VEEV,ADBE,QQQ,TSLA,W,ESTC,CLX,GDX,CDNS,MELI,ARKF,TTD,WK,SNAP,ESPO,NEM,YETI,CYTK,AAXN,FCX'


In [60]:
holdings=holdings.split(',')

In [68]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
holdings=['DVA']

In [69]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [70]:
holdings

['DVA']

In [71]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(22)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


DVA


In [65]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12]    
             }
            )

In [66]:
res.sort_values(by='k_change',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
18,UAA,2020-08-04,0.329211,-0.638732,-0.475587,4.11,44.83,29.02,0,18.45,0.869441,-0.062603,0.024530
20,NWL,2020-08-04,0.486885,-0.479496,-0.474012,9.74,51.91,51.47,0,17.92,1.866570,-0.026337,0.034126
24,URA,2020-08-03,0.479249,-0.436376,-0.575015,15.88,50.76,62.33,0,29.58,0.925007,-0.047645,0.057798
17,CAKE,2020-08-04,0.508535,-0.409659,-0.471870,12.58,53.19,55.23,0,68.89,2.940923,-0.018690,0.123609
23,DVA,2020-08-04,0.567481,-0.408748,-0.411877,10.22,48.37,34.98,0,11.33,4.747391,-0.003978,0.040716
38,TSLA,2020-08-04,0.454626,-0.276675,-0.449345,18.39,55.86,59.00,0,0.43,328.929203,-0.012055,0.214800
48,SNAP,2020-08-04,0.156995,-0.255815,-0.501647,21.27,44.12,34.98,1,2.20,4.198824,-0.017867,-0.059001
16,BATT,2020-08-03,0.738117,-0.228191,-0.707539,26.83,58.12,78.30,0,27.57,1.123172,0.003172,0.112939
54,FCX,2020-08-04,0.588615,-0.227104,-0.496053,23.24,52.03,50.93,0,13.69,2.170409,-0.027833,0.111739
14,EAT,2020-08-04,0.741566,-0.192788,-0.481101,26.05,59.43,74.29,1,95.54,5.160652,0.033032,0.253047


In [67]:
res.sort_values(by='22dayret',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
5,FUTU,2020-08-04,0.899851,0.202028,-0.489978,69.09,77.46,96.20,8,97.75,8.199939,0.218425,0.379595
3,FSLY,2020-08-04,0.923760,1.204288,-0.497101,91.46,76.10,89.42,7,97.72,27.704566,0.293301,0.335049
39,W,2020-08-04,0.967299,0.040522,-0.458633,55.96,75.92,96.29,7,99.32,58.881466,0.216150,0.330476
10,GDXJ,2020-08-04,0.832795,-0.012366,-0.538440,48.65,67.20,93.93,1,54.33,8.222864,0.002007,0.262060
14,EAT,2020-08-04,0.741566,-0.192788,-0.481101,26.05,59.43,74.29,1,95.54,5.160652,0.033032,0.253047
11,BGS,2020-08-04,0.834758,-0.040380,-0.465559,42.36,77.53,97.67,7,98.07,3.462493,0.081358,0.229569
38,TSLA,2020-08-04,0.454626,-0.276675,-0.449345,18.39,55.86,59.00,0,0.43,328.929203,-0.012055,0.214800
2,AAPL,2020-08-04,0.927717,0.642601,-0.427046,88.05,76.92,94.86,5,99.85,47.485559,0.127078,0.204691
42,GDX,2020-08-04,0.828438,0.007446,-0.526824,50.82,66.12,92.70,0,58.56,4.681716,0.003109,0.203345
25,ITB,2020-08-04,0.930930,0.053141,-0.485195,58.97,67.60,91.52,0,92.89,7.019181,0.029959,0.202584


In [ ]:
res[(res.RSI_percentile>90) & (res.k_change>-100)].sort_values(by='ret1wk_mean',ascending=False)

In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [ ]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [ ]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [ ]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

In [ ]:
date=list(response.json()['Time Series (Daily)'].keys())

In [ ]:
date.insert(0,response2.json()[0]['date'])

In [ ]:
date

In [ ]:
spy


In [ ]:
np.nanpercentile(spy.k_change,q=10)

In [ ]:
response.json()[0]

In [ ]:
pd.to_datetime('today')

In [ ]:
pd.Timestamp.today().date()

In [ ]:
pd.to_datetime(t[0]).date()

In [ ]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

In [ ]:
spy.adj_close.rolling(90).std().values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

In [72]:
spy=spy.set_index('Date')#['2020-07-01':]

In [73]:
spy['ret']=spy.adj_close/spy.adj_close.shift(5)-1

In [74]:
spy['ret22']=spy.adj_close/spy.adj_close.shift(22)-1

In [75]:
np.nanpercentile(spy.k_change,q=10),np.nanpercentile(spy.ret,q=10)

(-0.41245216217558994, -0.04241812563743522)

In [76]:
def each_percentile(x,val):
    return (x<val).sum()/len(x)

In [77]:
spy['ret_p']=spy.ret.apply(lambda x: each_percentile(spy.ret,x))

In [78]:
spy['ret22_p']=spy.ret.apply(lambda x: each_percentile(spy['ret22'],x))

In [79]:
spy['2020-07-01':]

,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret22,ret_p,ret22_p
Date,,,,,,,,,,,,,
2020-07-01,79.30,81.01,79.300,80.640,80.640,89.39,75.30,0.262361,2.916682,0.044830,0.003734,0.876508,0.670113
2020-07-02,81.50,82.68,80.790,81.050,81.050,89.39,75.30,0.353206,3.117764,0.045806,0.002350,0.882826,0.676623
2020-07-06,81.87,82.65,80.550,81.220,81.220,89.39,75.30,0.402413,1.830283,0.074907,0.018816,0.950986,0.798200
2020-07-07,80.61,82.96,80.510,81.990,81.990,89.39,75.30,0.434351,0.655546,0.061909,-0.004855,0.932989,0.747272
2020-07-08,82.10,82.25,80.060,81.910,81.910,89.39,75.30,0.454696,0.287341,0.035001,-0.072157,0.820984,0.614589
2020-07-09,82.04,82.43,78.650,79.740,79.740,86.63,75.30,0.445271,0.106501,-0.011161,-0.083448,0.312464,0.346736
2020-07-10,80.34,81.30,79.200,81.190,81.190,84.92,75.30,0.491091,0.130633,0.001727,-0.039172,0.479992,0.420639
2020-07-13,81.45,82.17,79.460,79.570,79.570,82.96,75.30,0.520529,0.144785,-0.020315,-0.040169,0.226690,0.295424
2020-07-14,79.56,82.11,79.220,81.950,81.950,82.96,75.30,0.679285,0.525554,-0.000488,0.025016,0.444189,0.407429


In [ ]:
spy['2020-02-20':'2020-3-31']


So something can go down at random 
look to see how other stocks have done. if it is aleader like CWH or MELI NEED TO track and buy
that is part of the in-cycle market leader

In [55]:
import requests
from bs4 import BeautifulSoup
import json

In [32]:
fetch_these=['Earnings','Market Cap','P/E','Forward P/E','Sales','Income','Dividend %','Short Float', 'Target Price'
             ,'ROE','Perf Week','Perf Month','Perf Quarter','Perf Year','Insider Own','Insider Trans']

In [33]:

def fetcher(soup, fetch_these,ticker):
    temp=[]
    temp.append(ticker)
    temp.append(pd.datetime.now())
    for i in fetch_these:
        x=soup.find('table', class_='snapshot-table2').find('td', string=i).find_next('td').text
        temp.append(x)
    return temp

In [34]:
stock_stats=[]
for j in holdings:
    try:
        url=str.format('https://finviz.com/quote.ashx?t={}',j)
        r=requests.get(url)
        soup=BeautifulSoup(r.text)
        stock_stats.append(fetcher(soup,fetch_these,j))
        time.sleep(np.random.randint(1,5))
        if np.random.uniform()<=0.3:
            time.sleep(4)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(j, exc_type, exc_tb.tb_lineno)

W <class 'AttributeError'> 7


/Users/rittikghosh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [35]:
stock_stats

[]

In [39]:
j='W'
url=str.format('https://finance.yahoo.com/quote/W?p=W&.tsrc=fin-srch')
r=requests.get(url)

In [41]:
soup=BeautifulSoup(r.text)

In [ ]:
https://cloud.iexapis.com/stable/stock/W/upcoming-earnings?token=pk_9469e57bc642462d905e4dc0003523ab
    http://q4finance.com/analysis/earning%20Drift/BA

In [42]:
url='http://q4finance.com/analysis/earning%20Drift/BA'
r=requests.get(url)

In [44]:
soup=BeautifulSoup(r.text)

In [46]:
soup.find('table', class_='table table-responsive img-fluid  table_text_nowrap').text

AttributeError: 'NoneType' object has no attribute 'text'

In [93]:
url='https://service.q4finance.com/analysis/earning_drift_data'
r=requests.post(url, data={'identifier':'AAPL'})

In [91]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2020-05-05
2020-02-28
2019-10-31
2019-08-01
2019-05-02
2019-02-22
2018-11-01
2018-08-02
2018-05-02


In [94]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01
